In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
myheaders = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip',
    'DNT': '1',
    'Connection': 'close'
}

In [ ]:
def get_data(pageno):
    url='https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageno)+'?ie=UTF8&pg=+'+str(pageno)
    r=requests.get(url,headers=myheaders)
    soup=BeautifulSoup(r.content,'html.parser')
    all=[]
    for d in soup.find_all('div',attrs={'class':'a-section a-spacing-none aok-relative'}):
        all1=[]
        link=d.find('a',attrs={'class':'a-link-normal'})
        url1='https://www.amazon.in'+link.get('href')
        name=d.find('span',attrs={'class':'zg-text-center-align'})
        book_name=name.find('img',alt=True)
        if book_name is not None:
            all1.append(book_name.get('alt'))
        else:
            all1.append('NaN')
        if link is not None:
            all1.append(url1)
        else:
            all1.append('NaN')
        all.append(all1)
    return all

In [ ]:
import pandas as pd
no_of_pages=2
result=[]
for i in range(1,no_of_pages+1):
    result.append(get_data(i))
flatten=lambda l: [item for sublist in l for item in sublist]
result_new=pd.DataFrame(flatten(result))
books_names=result_new.iloc[:,0]
list_url=result_new.iloc[:,1]
list_urls=list_url.tolist()
result_new

,0,1
0,The Psychology of Money,https://www.amazon.in/Psychology-Money-Morgan-...
1,My First Library: Boxset of 10 Board Books for...,https://www.amazon.in/My-First-Library-Boxset-...
2,Ikigai: The Japanese secret to a long and happ...,https://www.amazon.in/Ikigai-H%C3%A9ctor-Garc%...
3,400 Days,https://www.amazon.in/400-Days-Chetan-Bhagat/d...
4,Atomic Habits: The life-changing million copy ...,https://www.amazon.in/Atomic-Habits-James-Clea...
...,...,...
95,The Official Cambridge Guide To Ielts Student'...,https://www.amazon.in/Official-Cambridge-Guide...
96,How to Win Friends and Influence People,https://www.amazon.in/How-Win-Friends-Influenc...
97,Oswaal CBSE Sample Question Paper Class 12 Eng...,https://www.amazon.in/Oswaal-Sample-Question-E...
98,Think and Grow Rich,https://www.amazon.in/Think-Grow-Rich-Napoleon...


In [ ]:
list_urls

['https://www.amazon.in/Psychology-Money-Morgan-Housel/dp/9390166268/ref=zg_bs_books_1/260-4084635-1682801?_encoding=UTF8&psc=1&refRID=WM343RYPMA383Z06HWAV',
 'https://www.amazon.in/My-First-Library-Boxset-Board/dp/9387779262/ref=zg_bs_books_2/260-4084635-1682801?_encoding=UTF8&psc=1&refRID=WM343RYPMA383Z06HWAV',
 'https://www.amazon.in/Ikigai-H%C3%A9ctor-Garc%C3%ADa/dp/178633089X/ref=zg_bs_books_3/260-4084635-1682801?_encoding=UTF8&psc=1&refRID=WM343RYPMA383Z06HWAV',
 'https://www.amazon.in/400-Days-Chetan-Bhagat/dp/1542094089/ref=zg_bs_books_4/260-4084635-1682801?_encoding=UTF8&psc=1&refRID=WM343RYPMA383Z06HWAV',
 'https://www.amazon.in/Atomic-Habits-James-Clear/dp/1847941834/ref=zg_bs_books_5/260-4084635-1682801?_encoding=UTF8&psc=1&refRID=WM343RYPMA383Z06HWAV',
 'https://www.amazon.in/Rich-Dad-Poor-Middle-Updates/dp/1612680194/ref=zg_bs_books_6/260-4084635-1682801?_encoding=UTF8&psc=1&refRID=WM343RYPMA383Z06HWAV',
 'https://www.amazon.in/Word-Power-Made-Norman-Lewis/dp/0143424688/r

In [ ]:
def get_full_info(url):
    r=requests.get(url,headers=myheaders)
    soup=BeautifulSoup(r.content,'html.parser')
    all=[]
    outer_div=soup.find('div', attrs={'id': 'detailBullets_feature_div'})
    list_features=['Publisher', 'Language','Paperback', 'ISBN-10', 'ISBN-13', 'Weight', 'Dimensions']
    for feature in list_features:
        flag=False
        for d in outer_div.find_all('span',attrs={'class':'a-list-item'}):
            x=d.find('span',attrs={'class':'a-text-bold'})
            if x is not None:
                if feature in x.text:
                    all.append(d.find_all('span', recursive=False)[1].text)
                    flag=True
        if flag is False:
            all.append('NaN')
    reviews = soup.find_all('div', attrs={'data-hook': 'review-collapsed'})
    n=0
    for review in reviews:
        if review is not None:
            txt = review.find('span')
            if txt is not None:
                all.append(txt.text)
            else:
                all.append('NaN')
        n+=1
        if n==3:
            break
    return all

In [ ]:
books_data=[]
for url in list_urls:
    data=get_full_info(url)
    books_data.append(data)
books_data


[['Jaico Publishing House (1 September 2020); Jaico Publishing House, JA-1, Jash Chambers, Off Sir Phirozshah Mehtra Road, Fort, Mumbai, Maharashtra 400023 Phone: 022 4030 6767',
  'English',
  '252 pages',
  '9390166268',
  '978-9390166268',
  '220 g',
  '20 x 14 x 4 cm',
  "\n  The expectations from this book were sky high. And, does it deliver. The key lessons on optimism and pessimism and long term aside - the two things that states with me:1. Expectations change even slower than reality. Everyone sees the world they have seen it before.2. Everyone plays the game of investment differently. You need to define your game.This is a fantastic book. It's not a book about investments. It is a book about your mind, and mine and how it thinks about money, savings, expenditure and investments. And, it is brilliant. Pick it up now.\n",
  "\n  See guys this book is for those who have had a lot of Thier skin in the game of stock market and thought of creating wealth from it.First level of inves

In [ ]:
flatten=lambda l: [item for item in l]
df=pd.DataFrame(flatten(books_data), columns=['Publisher', 'Language', 'Paperback', 'ISBN-10', 'ISBN-13', 'Weight', 'Dimensions','Review 1','Review 2','Review 3'])
df


,Publisher,Language,Paperback,ISBN-10,ISBN-13,Weight,Dimensions,Review 1,Review 2,Review 3
0,Jaico Publishing House (1 September 2020); Jai...,English,252 pages,9390166268,978-9390166268,220 g,20 x 14 x 4 cm,\n The expectations from this book were sky h...,\n See guys this book is for those who have h...,\n This is a must read for the youngsters of ...
1,Wonder House Books; First edition (1 January 2...,English,NaN,9789387779266,978-9387779266,830 g,20 x 14 x 4 cm,\n I have well researched many books & conclu...,\n Delivery was on time packaging was excelle...,"\n It’s a cute collection, but I wasn’t expex..."
2,Random House UK; 1st edition (27 September 2017),English,NaN,178633089X,978-1786330895,247 g,13.6 x 2.3 x 18.4 cm,\n The book does a decent job of relating the...,\n Just read a back of book and its enough do...,\n I personally believe that having a purpose...
3,Westland (8 October 2021),English,352 pages,1542094089,978-1542094085,230 g,12.9 x 2.54 x 19.81 cm,"\n Decent story. However, the plot is thin an...",\n 3star because of same old model cleeches.....,\n Again a signed copy!! Can't wait to read😍😍\n
4,Random House Business Books (30 October 2018),English,288 pages,1847941834,978-1847941831,390 g,15.3 x 2.3 x 23.4 cm,\n I was aware of many concepts of this book ...,\n And I absolutely mean it what I said in He...,"\n Before starting this book, write down some..."
...,...,...,...,...,...,...,...,...,...,...
95,Cambridge University Press; 2014th edition (28...,English,NaN,9781107442771,978-1107442771,1 kg 210 g,20 x 14 x 4 cm,\n This book contains basic guidelines which ...,\n Everyone must buy the book who prepared fo...,\n This is 2014 edition and not 2018 as menti...
96,Rupa; First edition (20 May 2016),English,296 pages,8129140179,978-8129140173,159 g,10.8 x 1.65 x 17.78 cm,\n I will tell you what this book can change ...,\n Best book REALLY helpful and some thing we...,"\n Best thing about this book is,its writings..."
97,Oswaal Books and Learning Private Limited; New...,English,168 pages,9354637949,978-9354637940,260 g,28 x 5 x 20 cm,\n This book contains Chapter wise and Topic ...,\n English is one of the most scoring subject...,\n I found it really helpful for studies for ...
98,"Amazing Reads (1 January 2014); Amazing Reads,...",English,250 pages,8192910911,978-8192910918,310 g,13 x 1.5 x 20 cm,\n Title: THINK AND GROW RICHAuthor: NAPOLEON...,\n It took quarter of a century to accumulate...,\n When you face criticism in your life try t...


In [ ]:
df.insert(0,'Book Name',books_names,True)
df.to_csv('books_data1.csv')
df

,Book Name,Publisher,Language,Paperback,ISBN-10,ISBN-13,Weight,Dimensions,Review 1,Review 2,Review 3
0,The Psychology of Money,Jaico Publishing House (1 September 2020); Jai...,English,252 pages,9390166268,978-9390166268,220 g,20 x 14 x 4 cm,\n The expectations from this book were sky h...,\n See guys this book is for those who have h...,\n This is a must read for the youngsters of ...
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books; First edition (1 January 2...,English,NaN,9789387779266,978-9387779266,830 g,20 x 14 x 4 cm,\n I have well researched many books & conclu...,\n Delivery was on time packaging was excelle...,"\n It’s a cute collection, but I wasn’t expex..."
2,Ikigai: The Japanese secret to a long and happ...,Random House UK; 1st edition (27 September 2017),English,NaN,178633089X,978-1786330895,247 g,13.6 x 2.3 x 18.4 cm,\n The book does a decent job of relating the...,\n Just read a back of book and its enough do...,\n I personally believe that having a purpose...
3,400 Days,Westland (8 October 2021),English,352 pages,1542094089,978-1542094085,230 g,12.9 x 2.54 x 19.81 cm,"\n Decent story. However, the plot is thin an...",\n 3star because of same old model cleeches.....,\n Again a signed copy!! Can't wait to read😍😍\n
4,Atomic Habits: The life-changing million copy ...,Random House Business Books (30 October 2018),English,288 pages,1847941834,978-1847941831,390 g,15.3 x 2.3 x 23.4 cm,\n I was aware of many concepts of this book ...,\n And I absolutely mean it what I said in He...,"\n Before starting this book, write down some..."
...,...,...,...,...,...,...,...,...,...,...,...
95,The Official Cambridge Guide To Ielts Student'...,Cambridge University Press; 2014th edition (28...,English,NaN,9781107442771,978-1107442771,1 kg 210 g,20 x 14 x 4 cm,\n This book contains basic guidelines which ...,\n Everyone must buy the book who prepared fo...,\n This is 2014 edition and not 2018 as menti...
96,How to Win Friends and Influence People,Rupa; First edition (20 May 2016),English,296 pages,8129140179,978-8129140173,159 g,10.8 x 1.65 x 17.78 cm,\n I will tell you what this book can change ...,\n Best book REALLY helpful and some thing we...,"\n Best thing about this book is,its writings..."
97,Oswaal CBSE Sample Question Paper Class 12 Eng...,Oswaal Books and Learning Private Limited; New...,English,168 pages,9354637949,978-9354637940,260 g,28 x 5 x 20 cm,\n This book contains Chapter wise and Topic ...,\n English is one of the most scoring subject...,\n I found it really helpful for studies for ...
98,Think and Grow Rich,"Amazing Reads (1 January 2014); Amazing Reads,...",English,250 pages,8192910911,978-8192910918,310 g,13 x 1.5 x 20 cm,\n Title: THINK AND GROW RICHAuthor: NAPOLEON...,\n It took quarter of a century to accumulate...,\n When you face criticism in your life try t...
